<a href="https://colab.research.google.com/github/zzl-hyun/bigdata_analyst_cert_v2/blob/main/part2/ch6/ch6_ex_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert_v2/blob/main/part2/ch6/ch6_ex_classification.ipynb)

# 스스로 해봅시다

# 환자의 당요병 여부를 예측하시오 -> 분류다 그죠?
- label : Outcom(0:정상, 1: 당뇨)
- roc_auc 사용 <- predict_proba() 사용해라 그죠?

In [136]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/diabetes_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/diabetes_test.csv")

In [126]:
train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,118,58,36,94,33.3,0.261,23,0
1,3,173,78,39,185,33.8,0.970,31,1
2,4,109,64,44,99,34.8,0.905,26,1
3,9,57,80,37,0,32.8,0.096,41,0
4,2,129,0,0,0,38.5,0.304,41,0
...,...,...,...,...,...,...,...,...,...
609,3,163,70,18,105,31.6,0.268,28,1
610,2,174,88,37,120,44.5,0.646,24,1
611,7,142,90,24,480,30.4,0.128,43,1
612,2,100,54,28,105,37.8,0.498,24,0


In [127]:
# missing value
train.isnull().sum().sum()
test.isnull().sum().sum()

np.int64(0)

In [128]:
# outliner
train.describe()
# Pregnancies, SkinThickness, Insulin, DiabetesPedigreeFunction
test.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000
mean,4.064935,121.831169,68.064935,18.811688,77.474026,31.694156,0.457468,33.201299
std,3.311054,29.611163,21.297358,16.001131,130.046960,7.883533,0.350588,11.682275
min,0.000000,56.000000,0.000000,0.000000,0.000000,0.000000,0.085000,21.000000
25%,1.000000,99.000000,62.500000,0.000000,0.000000,27.500000,0.234250,24.000000
50%,3.000000,119.500000,70.000000,20.500000,0.000000,32.000000,0.334500,29.000000
75%,6.000000,139.750000,80.000000,32.000000,105.750000,36.100000,0.592750,40.000000
max,13.000000,197.000000,114.000000,50.000000,846.000000,52.300000,2.329000,69.000000


In [137]:
y_train = train.pop('Outcome')
train.shape

(614, 8)

In [139]:
# scale
from sklearn.preprocessing import StandardScaler, MinMaxScaler
cols = train.columns[:]

sc = MinMaxScaler()
train[cols] = sc.fit_transform(train[cols])
test[cols] = sc.transform(test[cols])
train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,0.222935,0.606330,0.568577,0.211809,0.417843,0.477905,0.169723,0.204180
std,0.199089,0.163601,0.154485,0.160845,0.440697,0.117570,0.139419,0.196486
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.058824,0.497487,0.512295,0.000000,0.000000,0.406855,0.071520,0.050000
50%,0.176471,0.582915,0.590164,0.232323,0.279612,0.478390,0.129590,0.133333
75%,0.352941,0.707286,0.655738,0.323232,0.998544,0.545455,0.235589,0.333333
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [91]:
# # Pregnancies, SkinThickness, Insulin, DiabetesPedigreeFunction
# print(train.shape)

# q3 = train['Pregnancies'].quantile(0.75)
# q1 = train['Pregnancies'].quantile(0.25)
# iqr = q3 - q1
# iqr
# # print((q3 + 1.5*iqr), (q1 - 1.5*iqr))
# train['Pregnancies'] = train['Pregnancies'].clip(lower=q1, upper=q3)
# print(train.shape)

(614, 8)
(614, 8)


In [92]:
# # Pregnancies, SkinThickness, Insulin, DiabetesPedigreeFunction
# print(train.shape)

# q3 = train['SkinThickness'].quantile(0.75)
# q1 = train['SkinThickness'].quantile(0.25)
# iqr = q3 - q1
# iqr
# # print((q3 + 1.5*iqr), (q1 - 1.5*iqr))
# train['SkinThickness'] = train['SkinThickness'].clip(lower=q1, upper=q3)

# print(train.shape)

(614, 8)
(614, 8)


In [138]:
# Pregnancies, SkinThickness, Insulin, DiabetesPedigreeFunction
print(train.shape)

q3 = train['Insulin'].quantile(0.75)
q1 = train['Insulin'].quantile(0.25)
iqr = q3 - q1
iqr
# print((q3 + 1.5*iqr), (q1 - 1.5*iqr))
train['Insulin'] = train['Insulin'].clip(lower=q1, upper=q3)

print(train.shape)

(614, 8)
(614, 8)


In [94]:
# # Pregnancies, SkinThickness, Insulin, DiabetesPedigreeFunction
# print(train.shape)

# q3 = train['DiabetesPedigreeFunction'].quantile(0.75)
# q1 = train['DiabetesPedigreeFunction'].quantile(0.25)
# iqr = q3 - q1
# iqr
# # print((q3 + 1.5*iqr), (q1 - 1.5*iqr))
# train['DiabetesPedigreeFunction'] = train['DiabetesPedigreeFunction'].clip(lower=q1, upper=q3)

# print(train.shape)

(614, 8)
(614, 8)


In [109]:
train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.789902,120.659609,69.366450,20.969055,53.797231,32.067427,0.475490,33.250814
std,3.384519,32.556686,18.847224,15.923677,56.739803,7.888971,0.326519,11.789160
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.500000,0.000000,0.000000,27.300000,0.245500,24.000000
50%,3.000000,116.000000,72.000000,23.000000,36.000000,32.100000,0.381500,29.000000
75%,6.000000,140.750000,80.000000,32.000000,128.562500,36.600000,0.629750,41.000000
max,17.000000,199.000000,122.000000,99.000000,128.750000,67.100000,2.420000,81.000000


In [140]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2, random_state=0)


In [151]:
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm

rf = RandomForestClassifier(random_state=0, max_depth=5, n_estimators=500)
lgbmc = lgbm.LGBMClassifier(random_state=0, learning_rate=0.05, n_estimators=1000, max_depth=5, num_leaves=31)

rf.fit(X_train, y_train)
lgbmc.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 161, number of negative: 330
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000100 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 542
[LightGBM] [Info] Number of data points in the train set: 491, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.327902 -> initscore=-0.717688
[LightGBM] [Info] Start training from score -0.717688
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

LGBMClassifier(learning_rate=0.05, max_depth=5, n_estimators=1000,
               random_state=0)

In [152]:
from sklearn.metrics import roc_auc_score
pred_rf = rf.predict_proba(X_val)
pred_lgbmc = lgbmc.predict_proba(X_val)
roc_rf = roc_auc_score(y_val, pred_rf[:,1])
roc_lgbmc = roc_auc_score(y_val, pred_lgbmc[:,1])
# pred_rf[:,1]
print(roc_rf, roc_lgbmc)

0.82986301369863 0.8090410958904111


In [123]:
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred': pred[:,1]})
submit.to_csv('result.csv', index=False)
pd.read_csv('result.csv')

,pred
0,0.06
1,0.06
2,0.08
3,0.06
4,0.06
...,...
149,0.06
150,0.10
151,0.06
152,0.06


# Section1

### 베이스라인

In [ ]:
# 1. 문제정의
# 평가: roc-auc
# target: Outcome
# 최종파일: result.csv(컬럼 1개 pred, 1확률값)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("diabetes_train.csv")
# test = pd.read_csv("diabetes_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/diabetes_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/diabetes_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
print("\n") # 줄 바꿈

print("===== train 데이터 샘플 =====")
print(train.head(1))
print("\n")

print("===== test 데이터 샘플 =====")
print(test.head(1))
print("\n")

print("===== 데이터 정보(자료형) =====")
print(train.info())
print("\n")

print("===== train 결측치 수 =====")
print(train.isnull().sum().sum())
print("\n")

print("===== test 결측치 수 =====")
print(test.isnull().sum().sum())
print("\n")

print("===== target 빈도 =====")
print(train['Outcome'].value_counts())

===== 데이터 크기 =====
Train Shape: (614, 9)
Test Shape: (154, 8)


===== train 데이터 샘플 =====
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            1      118             58             36       94  33.3   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.261   23        0  


===== test 데이터 샘플 =====
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            3      102             74              0        0  29.5   

   DiabetesPedigreeFunction  Age  
0                     0.121   32  


===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    int64  
 2   BloodPressure             614 non-null    int64  
 3   SkinThickness             6

In [ ]:
# 4. 데이터 전처리
target = train.pop('Outcome')

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

print("\n ===== 분할된 데이터 크기 =====")
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)

print("\n ===== 예측 결과 확인 (샘플 5개) =====")
print(pred[:5])

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('\n roc_auc:', roc_auc)

# 7. 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

# 제출파일 확인
print("\n ===== 제출파일 (샘플 5개) =====")
print(pd.read_csv("result.csv").head())


 ===== 분할된 데이터 크기 =====
(491, 8) (123, 8) (491,) (123,)

 ===== 예측 결과 확인 (샘플 5개) =====
[[0.88 0.12]
 [0.29 0.71]
 [0.41 0.59]
 [0.75 0.25]
 [0.89 0.11]]

 roc_auc: 0.8002739726027398

 ===== 제출파일 (샘플 5개) =====
   pred
0  0.17
1  0.33
2  0.11
3  0.04
4  0.09


### 성능개선

In [ ]:
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("diabetes_train.csv")
# test = pd.read_csv("diabetes_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/diabetes_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/diabetes_test.csv")

# 4. 데이터 전처리
# 스케일링
target = train.pop('Outcome')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=5, n_estimators=500, random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

# 7. 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

# 스케일링: MinMaxScaler 0.8031506849315069 (StandardScaler, RobustScaler 대입)
# max_depth=5: 0.8153424657534246 (3, 5, 7 대입)
# n_estimators=500: 0.8246575342465753 (200, 400, 500 대입)

roc_auc: 0.8246575342465753


# Section2

### 베이스라인

In [ ]:
# 1. 문제정의
# 평가: roc-auc
# target: target
# 최종파일: result.csv(컬럼 1개 pred, 1확률값)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("hr_train.csv")
# test = pd.read_csv("hr_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/hr_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/hr_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum())

print("\n ===== train/test 카테고리별 수 =====")
print(train.nunique())
print(test.nunique())

print("\n ===== target 빈도 =====")
print(train['target'].value_counts())

===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  object 
 2   city_development_index  15326 non-null  float64
 3   gender                  11750 non-null  object 
 4   relevent_experience     15326 non-null  object 
 5   enrolled_university     15012 non-null  object 
 6   education_level         14961 non-null  object 
 7   major_discipline        13045 non-null  object 
 8   experience              15272 non-null  object 
 9   company_size            10539 non-null  object 
 10  company_type            10383 non-null  object 
 11  last_new_job            14984 non-null  object 
 12  training_hours          15326 non-null  int64  
 13  target                  15326 non-null  float64
dtypes: float64(2),

In [ ]:
# train = pd.read_csv("hr_train.csv")
# test = pd.read_csv("hr_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/hr_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/hr_test.csv")
cols = train.select_dtypes(include='object').columns
for col in cols:
    set_train = set(train[col])
    set_test= set(test[col])
    same = (set_train == set_test)
    if same:
        print(col, "\t카테고리 동일함")
    else:
        print(col, "\t카테고리 동일하지 않음")

city 	카테고리 동일하지 않음
gender 	카테고리 동일함
relevent_experience 	카테고리 동일함
enrolled_university 	카테고리 동일함
education_level 	카테고리 동일함
major_discipline 	카테고리 동일함
experience 	카테고리 동일함
company_size 	카테고리 동일함
company_type 	카테고리 동일함
last_new_job 	카테고리 동일함


In [ ]:
# 4. 데이터 전처리
target = train.pop('target')

# 결측치 처리
train = train.fillna("X")
test = test.fillna("X")

# train과 test 합쳐서 원핫인코딩
combined = pd.concat([train, test])
combined_dummies = pd.get_dummies(combined)
n_train = len(train)
train = combined_dummies[:n_train]
test = combined_dummies[n_train:]

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

# 7. 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)


roc_auc: 0.7730742036233207


### 성능개선

In [ ]:
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("hr_train.csv")
# test = pd.read_csv("hr_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/hr_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/hr_test.csv")

# 4. 데이터 전처리
target = train.pop('target')

# 결측치 처리
train = train.fillna("X")
test = test.fillna("X")

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
combined = pd.concat([train, test])
cols = train.select_dtypes(include='object').columns

for col in cols:
    le = LabelEncoder()
    combined[col] = le.fit_transform(combined[col])

n_train = len(train)
train = combined[:n_train]
test = combined[n_train:]

# id 제거 (성능 떨어짐)
# train = train.drop('enrollee_id', axis=1)
# test = test.drop('enrollee_id', axis=1)

# 스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
n_cols = train.select_dtypes(exclude='object').columns
train = scaler.fit_transform(train)
test = scaler.transform(test)

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=7, n_estimators=200, random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)

from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred[:,1])
print('roc_auc:', roc_auc)

# 7. 예측 및 결과 파일 생성
pred = rf.predict_proba(test)
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

roc_auc: 0.7825363713412095


# Section3

### 베이스라인

In [ ]:
# 1. 문제정의
# 평가: f1
# target: STATUS
# 최종파일: result.csv(컬럼 1개 pred)

# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("creditcard_train.csv")
# test = pd.read_csv("creditcard_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/creditcard_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/creditcard_test.csv")

# 3. 탐색적 데이터 분석(EDA)
print("===== 데이터 크기 =====")
print(train.shape, test.shape)

print("\n ===== 데이터 정보(자료형) =====")
print(train.info())

print("\n ===== train 결측치 수 =====")
print(train.isnull().sum())

print("\n ===== test 결측치 수 =====")
print(test.isnull().sum())

print("\n ===== 범주형 테이터 카테고리 =====")
cols = train.select_dtypes(include='object').columns
for col in cols:
    set_train = set(train[col])
    set_test= set(test[col])
    same = (set_train == set_test)
    if same:
        print(col, "\t카테고리 동일함")
    else:
        print(col, "\t카테고리 동일하지 않음")

print("\n ===== target 빈도 =====")
print(train['STATUS'].value_counts())

===== 데이터 크기 =====
(25519, 19) (7591, 18)

 ===== 데이터 정보(자료형) =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25519 entries, 0 to 25518
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   25519 non-null  int64  
 1   CODE_GENDER          25519 non-null  object 
 2   FLAG_OWN_CAR         25519 non-null  object 
 3   FLAG_OWN_REALTY      25519 non-null  object 
 4   CNT_CHILDREN         25519 non-null  int64  
 5   AMT_INCOME_TOTAL     25519 non-null  float64
 6   NAME_INCOME_TYPE     25519 non-null  object 
 7   NAME_EDUCATION_TYPE  25519 non-null  object 
 8   NAME_FAMILY_STATUS   25519 non-null  object 
 9   NAME_HOUSING_TYPE    25519 non-null  object 
 10  DAYS_BIRTH           25519 non-null  int64  
 11  DAYS_EMPLOYED        25519 non-null  int64  
 12  FLAG_MOBIL           25519 non-null  int64  
 13  FLAG_WORK_PHONE      25519 non-null  int64  
 14  FLAG_PHONE        

In [ ]:
# 4. 데이터 전처리
# 결측치 처리
print("삭제 전:",train.shape)
train.dropna(subset=['OCCUPATION_TYPE'], inplace=True)
print("삭제 후:",train.shape)

target = train.pop('STATUS')

# 원핫인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
score = f1_score(y_val, pred)
print('\n f1:', score)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

삭제 전: (25519, 19)
삭제 후: (17543, 19)

 f1: 0.22018348623853212


### 성능개선

In [ ]:
# 2. 라이브러리 및 데이터 불러오기
import pandas as pd

# train = pd.read_csv("creditcard_train.csv")
# test = pd.read_csv("creditcard_test.csv")
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/creditcard_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch6/creditcard_test.csv")

# 4. 데이터 전처리
# 결측치 처리 (최빈값)
freq = train['OCCUPATION_TYPE'].mode()[0]
train['OCCUPATION_TYPE'] = train['OCCUPATION_TYPE'].fillna(freq)
test['OCCUPATION_TYPE'] = test['OCCUPATION_TYPE'].fillna(freq)

target = train.pop('STATUS')

# 스케일링 (성능 개선 효과 없음)
# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()
# n_cols = train.select_dtypes(exclude='object').columns[:-1] # STATUS를 제외한 int, float
# train[n_cols] = scaler.fit_transform(train[n_cols])
# test[n_cols] = scaler.transform(test[n_cols])

# ID 제외
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

# 5. 검증 데이터 분할
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 6. 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=0)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)

from sklearn.metrics import f1_score
score = f1_score(y_val, pred)
print('f1:', score)

# 7. 예측 및 결과 파일 생성
pred = rf.predict(test)
submit = pd.DataFrame({'pred':pred})
submit.to_csv("result.csv", index=False)

f1: 0.3253968253968254
